## HW4

1. Import the Iris dataset.
2. Build a K-means model to classify the species of Iris. You can choose a k value randomly
at this step.
3. Report the original performance using Silhouette score.
4. Try to improve the performance of the original model by trying at least 10 different k
values.
5. Select the best k based on step 4 and print out the following sentence in your code:
“k=xx gives the best performance, Silhouette =xx "
(replace xx with your own numbers) \\

A tutorial of spark can be found at:
https://spark.apache.org/docs/latest/ml-clustering.html

### Install Spark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Get Spark installer (check the path on spark.apache.org)
!wget -q http://apache.mirrors.pair.com/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz

#Untar the Spark installer
!tar -xvf spark-2.4.7-bin-hadoop2.7.tgz

# Install findspark - a python library to find Spark
!pip install -q findspark

# Set environment variables
# Set Java and Spark home based on the location where they are stored
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

# Create a local Spark session
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
# Import the package
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
import string
import re
import nltk

# Create SparkContextz
sc = SparkContext.getOrCreate()
print("SparkContext : ", sc)

# Define spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

SparkContext :  <SparkContext master=local[*] appName=pyspark-shell>


### 1. Import the data + connect to drive


In [ ]:
# Mount with gdrive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
os.chdir("/content/gdrive/My Drive/IDS561HW4")
!ls

iris.data


In [ ]:
#  Loading CSV file to dataframe
# import the data
import pandas as pd
data = pd.read_csv("/content/gdrive/MyDrive/IDS561HW4/iris.data", header=None)

# adding column name to the respective columns
data.columns =['sepal_length','sepal_width','petal_length','petal _width','class']

# add a row index as a string
data['id'] = 'row'+data.index.astype(str)

# move it first (left)
cols = list(data)
cols.insert(0, cols.pop(cols.index('id')))
data = data.loc[:, cols]
data.head()


,id,sepal_length,sepal_width,petal_length,petal _width,class
0,row0,5.1,3.5,1.4,0.2,Iris-setosa
1,row1,4.9,3.0,1.4,0.2,Iris-setosa
2,row2,4.7,3.2,1.3,0.2,Iris-setosa
3,row3,4.6,3.1,1.5,0.2,Iris-setosa
4,row4,5.0,3.6,1.4,0.2,Iris-setosa


In [ ]:
# Create a Spark DataFrame from a Pandas DataFrame using Arrow
df = spark.createDataFrame(data)
df.show(5)

+----+------------+-----------+------------+------------+-----------+
|  id|sepal_length|sepal_width|petal_length|petal _width|      class|
+----+------------+-----------+------------+------------+-----------+
|row0|         5.1|        3.5|         1.4|         0.2|Iris-setosa|
|row1|         4.9|        3.0|         1.4|         0.2|Iris-setosa|
|row2|         4.7|        3.2|         1.3|         0.2|Iris-setosa|
|row3|         4.6|        3.1|         1.5|         0.2|Iris-setosa|
|row4|         5.0|        3.6|         1.4|         0.2|Iris-setosa|
+----+------------+-----------+------------+------------+-----------+
only showing top 5 rows



2. Build a K-means model to classify the species of Iris. You can choose a k value randomly at this step.

In [ ]:
# Import library
from __future__ import print_function

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.datasets.samples_generator import make_blobs
from pyspark import SparkContext
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SQLContext

%matplotlib inline

print (pyspark.__version__)

2.4.7


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
# https://rsandstroem.github.io/sparkkmeans.html
# Make features column
FEATURES_COL = ["sepal_length", "sepal_width","petal_length","petal_width"]
for col in df.columns:
    if col in FEATURES_COL:
        df = df.withColumn(col,df[col].cast('float'))
df.show(5)


+----+------------+-----------+------------+------------+-----------+
|  id|sepal_length|sepal_width|petal_length|petal _width|      class|
+----+------------+-----------+------------+------------+-----------+
|row0|         5.1|        3.5|         1.4|         0.2|Iris-setosa|
|row1|         4.9|        3.0|         1.4|         0.2|Iris-setosa|
|row2|         4.7|        3.2|         1.3|         0.2|Iris-setosa|
|row3|         4.6|        3.1|         1.5|         0.2|Iris-setosa|
|row4|         5.0|        3.6|         1.4|         0.2|Iris-setosa|
+----+------------+-----------+------------+------------+-----------+
only showing top 5 rows



In [ ]:
#https://community.intersystems.com/post/k-means-clustering-iris-dataset

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = ["sepal_length", "sepal_width","petal_length", "petal _width"],
                            outputCol="features") #it makes a vector with 4 parameters mentioned in inputCols and name it as outputCol.

irisFeatures = assembler.transform(df) # this will add to the table outputCol column with vectors.
irisFeatures.show(5)

+----+------------+-----------+------------+------------+-----------+--------------------+
|  id|sepal_length|sepal_width|petal_length|petal _width|      class|            features|
+----+------------+-----------+------------+------------+-----------+--------------------+
|row0|         5.1|        3.5|         1.4|         0.2|Iris-setosa|[5.09999990463256...|
|row1|         4.9|        3.0|         1.4|         0.2|Iris-setosa|[4.90000009536743...|
|row2|         4.7|        3.2|         1.3|         0.2|Iris-setosa|[4.69999980926513...|
|row3|         4.6|        3.1|         1.5|         0.2|Iris-setosa|[4.59999990463256...|
|row4|         5.0|        3.6|         1.4|         0.2|Iris-setosa|[5.0,3.5999999046...|
+----+------------+-----------+------------+------------+-----------+--------------------+
only showing top 5 rows



In [ ]:
# Install and import package: KMeans, ClusteringEvaluator
# https://towardsdatascience.com/k-means-clustering-using-pyspark-on-big-data-6214beacdc8b
# https://medium.com/swlh/k-means-clustering-using-pyspark-on-data-bricks-fd65e207154a

from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Train a k-means model with k = 3
kmeans = KMeans().setK(3).setSeed(101)
model = kmeans.fit(irisFeatures)
predictions = model.transform(irisFeatures)
#predictions.show(15)

3. Report the original performance using Silhouette score. 

In [ ]:
# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print("Silhouette Score = " + str(silhouette))

Silhouette Score = 0.7342112983712945


4. Try to improve the performance of the original model by trying at least 10 different k values.


In [ ]:
# Try different 10 values of K 
# set initial parameter
silhouette_max = 0
imax = 0 

for i in range(2,12):
  # Train a k-means model
  kmeans = KMeans().setK(i).setSeed(101)
  model = kmeans.fit(irisFeatures)
  predictions = model.transform(irisFeatures)

  # Evaluate clustering by computing Silhouette score
  evaluator = ClusteringEvaluator()
  silhouette = evaluator.evaluate(predictions)
  print("K ="+ str(i) + " Silhouette Score = " + str(silhouette))

  if silhouette > silhouette_max:
    silhouette_max = silhouette
    imax = i



K =2 Silhouette Score = 0.8501515988887648
K =3 Silhouette Score = 0.7342112983712945
K =4 Silhouette Score = 0.6786090224788793
K =5 Silhouette Score = 0.5642107559033724
K =6 Silhouette Score = 0.6451259926752204
K =7 Silhouette Score = 0.522040093038452
K =8 Silhouette Score = 0.48449196173363857
K =9 Silhouette Score = 0.4700162563790132
K =10 Silhouette Score = 0.4924102836125928
K =11 Silhouette Score = 0.4239762948562177


5. Select the best k based on step 4 and print out the following sentence in your code:
“k=xx gives the best performance, Silhouette =xx " (replace xx with your own numbers)"

In [ ]:
print("k="+ str(imax) + " gives the best performance, Silhouette ="+ str(silhouette_max))

k=2 gives the best performance, Silhouette =0.8501515988887648
